In [1]:
# import libaries

from PIL import Image
from tqdm.notebook import tqdm
import numpy as np
import random
import sys
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchinfo import summary
from torchvision import transforms
from torch import optim
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import KFold

In [2]:
print('Python version:', sys.version)
print('CUDA Available:', torch.cuda.is_available())

if torch.cuda.is_available():
    print('GPU Name:', torch.cuda.get_device_name())
    print('GPU Properties:\n', torch.cuda.get_device_properties('cuda'))
    device = "cuda"
    torch.cuda.set_per_process_memory_fraction(0.95, 0)
    torch.cuda.empty_cache()
else:
    print("Cuda is not available, please use cpu instead")
    device = "cpu"
!nvidia-smi

Python version: 3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)]
CUDA Available: True
GPU Name: NVIDIA GeForce RTX 2070
GPU Properties:
 _CudaDeviceProperties(name='NVIDIA GeForce RTX 2070', major=7, minor=5, total_memory=8191MB, multi_processor_count=36)
Sat Feb 18 17:31:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.98       Driver Version: 526.98       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   43C    P2    55W / 175W |    316Mi

In [3]:
seed = 41
# Define custom dataset
class FaceDataset(Dataset):
    def __init__(self, df = None, num_sample = None, transform = None, num_img_pool = 10):
        # set random seed for FaceDataset
        np.random.seed(seed)
        random.seed(seed)
        # create constructors
        self.unique_img_name = None
        self.data = dict()
        self.images = list()
        # label to indices
        self.label_to_indices = dict()
        self.labels = list()
        # read csv file
        self.df = df
        # set the transformation
        self.transform = transform
        # drop last n row from dataframe
        self.df = self.df.head(num_sample)
        #get the length of entire dataset
        self.len_ = len(self.df)
        # load imgs
        self.load_imgs(self.df, num_imgs = num_img_pool, max = num_sample)

    def __len__(self):
        return self.len_

    # get each pair of images -> 1: same identity, 0: different identity
    # if index is even -> same pair
    # if index is odd -> random identity
    def __getitem__(self, idx):
        anchor_img = self.images[idx]
        anchor_label = self.labels[idx]

        pos_idx = np.random.choice(np.arange(len(self.images))[self.labels == anchor_label])
        neg_idx = np.random.choice(np.arange(len(self.images))[self.labels != anchor_label])

        pos_img = self.images[pos_idx]
        neg_img = self.images[neg_idx]

        pos_label = self.labels[pos_idx]
        neg_label = self.labels[neg_idx]

        if self.transform is None:
            img_to_tensor = transforms.ToTensor()
            anchor_img = img_to_tensor(anchor_img)
            pos_img = img_to_tensor(pos_img)
            neg_img = img_to_tensor(neg_img)
        else:
            anchor_img = self.transform(anchor_img)
            pos_img = self.transform(pos_img)
            neg_img = self.transform(neg_img)

        return anchor_img, pos_img, neg_img

    # load imgs from pandas to memory and define the maximum number of images
    def load_imgs(self, df, num_imgs, max):
        # iterate thought each row
        for i, row in tqdm(df.iterrows(), total = max):
            # get identity of each row
            row_identity = row['identity']
            # append each identity to numberical value
            self.label_to_indices[int(row_identity)] = i
            count_img = 0
            # loop imgs in each identity
            for img_name in row['path']:
                if count_img > num_imgs:
                    break
                # concatenate the directoru and image name
                # path_to_image = self.dir+img_name
                path_to_image = img_name
                # open image and convert to RGB
                img = Image.open(path_to_image).convert('RGB')

                self.images.append(img)
                self.labels.append(i)
                count_img += 1
            # print('Added img '+ str(row_identity))
        self.labels = np.array(self.labels)

In [4]:
ds_df = pd.read_csv('./digiface_csv_files/digi_all.csv')
ds_df = ds_df.groupby('identity')['path'].apply(list).reset_index()
ds_df

,identity,path
0,0,"[digiFace1M\subjects_0-1999_72_imgs\0\20.png, ..."
1,1,"[digiFace1M\subjects_0-1999_72_imgs\1\66.png, ..."
2,2,"[digiFace1M\subjects_0-1999_72_imgs\2\29.png, ..."
3,3,"[digiFace1M\subjects_0-1999_72_imgs\3\42.png, ..."
4,4,"[digiFace1M\subjects_0-1999_72_imgs\4\33.png, ..."
...,...,...
72661,199994,[digiFace1M\subjects_166666-199998_5_imgs\1999...
72662,199995,[digiFace1M\subjects_166666-199998_5_imgs\1999...
72663,199996,[digiFace1M\subjects_166666-199998_5_imgs\1999...
72664,199997,[digiFace1M\subjects_166666-199998_5_imgs\1999...


In [5]:
from sklearn.model_selection import train_test_split

# splitting each dataset
train_df, eval_df = train_test_split(ds_df, test_size= 0.3, shuffle = True, random_state = seed)
val_df, test_df = train_test_split(eval_df, test_size = 0.4, shuffle = True, random_state = seed)

# print to check size of each dataset
print(f'Train Size: {len(train_df)}')
print(f'Val Size: {len(val_df)}')
print(f'Test Size: {len(test_df)}')

train_df.head(5)

Train Size: 50866
Val Size: 13080
Test Size: 8720


,identity,path
26344,120344,[digiFace1M\subjects_100000-133332_5_imgs\1203...
61833,189166,[digiFace1M\subjects_166666-199998_5_imgs\1891...
46921,174254,[digiFace1M\subjects_166666-199998_5_imgs\1742...
4873,8873,[digiFace1M\subjects_8000-9999_72_imgs\8873\54...
19875,113875,[digiFace1M\subjects_100000-133332_5_imgs\1138...


In [6]:
# define image size
img_size = 112

# # define transformation for test set
# train_transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.CenterCrop(img_size),
#     transforms.RandomHorizontalFlip(p=0.6),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])


# # define transformation for validation set
# val_transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.CenterCrop(img_size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])


# # define batch size
# train_batch_size = 64
# val_batch_size = 64
# print('------------Started Loading Train Set------------')
# # create dataloader for train set
# train_triplet_dataset = FaceDataset(df = train_df, num_sample = 6000, transform = train_transform)
# train_triplet_dataloader = DataLoader(train_triplet_dataset, batch_size=train_batch_size, shuffle=True, pin_memory=True)
# print('Total Train Set: ', train_triplet_dataset.__len__())
# print('-----------Finished Loading Train Set------------')

# print('\n')

# print('------------Started Loading Validation Set------------')
# # create dataloader for validation set
# val_triplet_dataset = FaceDataset(df = val_df,num_sample = 3000, transform = val_transform)
# val_triplet_dataloader = DataLoader(val_triplet_dataset, batch_size=val_batch_size, shuffle=True, pin_memory=True)
# print('Total Train Set: ', val_triplet_dataset.__len__())
# print('-----------Finished Loading Validation Set------------')


In [7]:
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, ReLU6, Sigmoid, Dropout2d\
    ,Dropout, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Sequential, Module, Parameter

class Flatten(Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

def l2_norm(input,axis=1):
    norm = torch.norm(input,2,axis,True)
    output = torch.div(input, norm)
    return output

class h_sigmoid(Module):
    def __init__(self, inplace=True):
        super(h_sigmoid, self).__init__()
        self.relu = ReLU6(inplace=inplace)

    def forward(self, x):
        return self.relu(x + 3) / 6


class h_swish(Module):
    def __init__(self, inplace=True):
        super(h_swish, self).__init__()
        self.sigmoid = h_sigmoid(inplace=inplace)

    def forward(self, x):
        return x * self.sigmoid(x)


class SELayer(Module):
    def __init__(self, channel, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = AdaptiveAvgPool2d(1)
        self.fc = Sequential(
            Linear(channel, channel // reduction),
            ReLU(inplace=True),
            Linear(channel // reduction, channel),
            h_sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

class PermutationBlock(Module):
    def __init__(self, groups):
        super(PermutationBlock, self).__init__()
        self.groups = groups

    def forward(self, input):
        n, c, h, w = input.size()
        G = self.groups
        output = input.view(n, G, c // G, h, w).permute(0, 2, 1, 3, 4).contiguous().view(n, c, h, w)
        return output

class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x

class Depth_Wise(Module):
    def __init__(self, in_c, out_c, residual = False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        self.conv = Conv_block(in_c, out_c=groups, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(groups, groups, groups=groups, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(groups, out_c, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual
    def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output

class Residual(Module):
    def __init__(self, c, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for _ in range(num_block):
            modules.append(Depth_Wise(c, c, residual=True, kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)
    def forward(self, x):
        return self.model(x)

class LinearScheduler(nn.Module):
    def __init__(self, dropblock, start_value, stop_value, nr_steps):
        super(LinearScheduler, self).__init__()
        self.dropblock = dropblock
        self.i = 0
        self.drop_values = np.linspace(start=start_value, stop=stop_value, num=nr_steps)

    def forward(self, x):
        return self.dropblock(x)

    def step(self):
        if self.i < len(self.drop_values):
            self.dropblock.drop_prob = self.drop_values[self.i]

        self.i += 1

class DropBlock2D(nn.Module):
    r"""Randomly zeroes 2D spatial blocks of the input tensor.
    As described in the paper
    `DropBlock: A regularization method for convolutional networks`_ ,
    dropping whole blocks of feature map allows to remove semantic
    information as compared to regular dropout.
    Args:
        drop_prob (float): probability of an element to be dropped.
        block_size (int): size of the block to drop
    Shape:
        - Input: `(N, C, H, W)`
        - Output: `(N, C, H, W)`
    .. _DropBlock: A regularization method for convolutional networks:
       https://arxiv.org/abs/1810.12890
    """

    def __init__(self, drop_prob, block_size):
        super(DropBlock2D, self).__init__()

        self.drop_prob = drop_prob
        self.block_size = block_size

    def forward(self, x):
        # shape: (bsize, channels, height, width)

        assert x.dim() == 4, \
            "Expected input with 4 dimensions (bsize, channels, height, width)"

        if not self.training or self.drop_prob == 0.:
            return x
        else:
            # get gamma value
            gamma = self._compute_gamma(x)

            # sample mask
            mask = (torch.rand(x.shape[0], *x.shape[2:]) < gamma).float()

            # place mask on input device
            mask = mask.to(x.device)

            # compute block mask
            block_mask = self._compute_block_mask(mask)

            # apply block mask
            out = x * block_mask[:, None, :, :]

            # scale output
            out = out * block_mask.numel() / block_mask.sum()

            return out

    def _compute_block_mask(self, mask):
        block_mask = F.max_pool2d(input=mask[:, None, :, :],
                                  kernel_size=(self.block_size, self.block_size),
                                  stride=(1, 1),
                                  padding=self.block_size // 2)

        if self.block_size % 2 == 0:
            block_mask = block_mask[:, :, :-1, :-1]

        block_mask = 1 - block_mask.squeeze(1)

        return block_mask

    def _compute_gamma(self, x):
        return self.drop_prob / (self.block_size ** 2)


In [8]:

class MobileFaceNet(Module):
    def __init__(self, embedding_size=512):
        super(MobileFaceNet, self).__init__()
        self.conv1 = Conv_block(3, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(64, 64, kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        self.conv_23 = Depth_Wise(64, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
        self.conv_3 = Residual(64, num_block=4, groups=128, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_34 = Depth_Wise(64, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        self.conv_4 = Residual(128, num_block=6, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_45 = Depth_Wise(128, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=512)
        self.conv_5 = Residual(128, num_block=2, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_6_sep = Conv_block(128, 512, kernel=(1, 1), stride=(1, 1), padding=(0, 0))
        self.conv_6_dw = Linear_block(512, 512, groups=512, kernel=(7,7), stride=(1, 1), padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(512, embedding_size, bias=False)
        self.bn = BatchNorm1d(embedding_size)
        self.dropblock = DropBlock2D(block_size=3, drop_prob=0.3)
        self.dropout = Dropout2d(0.1)

    def forward_once(self,x):
        out = self.conv1(x)

        out = self.conv2_dw(out)

        out = self.conv_23(out)

        out = self.conv_3(out)

        out = self.conv_34(out)

        out = self.conv_4(out)

        # out = self.dropblock(out)

        out = self.conv_45(out)

        out = self.conv_5(out)

        out = self.conv_6_sep(out)

        out = self.dropblock(out)

        out = self.conv_6_dw(out)

        out = self.dropblock(out)

        out = self.conv_6_flatten(out)

        out = self.linear(out)

        out = self.bn(out)
        return l2_norm(out)

    def forward(self, anchor_img, positive_img, negative_img):
        anchor = self.forward_once(anchor_img)
        positive = self.forward_once(positive_img)
        negative = self.forward_once(negative_img)
        return anchor, positive, negative


In [9]:
# triplet_model = TripletNetwork()
triplet_model = MobileFaceNet()
print(summary(triplet_model, input_size=[(32,3,112,112),(32,3,112,112),(32,3,112,112)]))

Layer (type:depth-idx)                        Output Shape              Param #
MobileFaceNet                                 [32, 512]                 --
├─Conv_block: 1-1                             [32, 64, 56, 56]          --
│    └─Conv2d: 2-1                            [32, 64, 56, 56]          1,728
│    └─BatchNorm2d: 2-2                       [32, 64, 56, 56]          128
│    └─PReLU: 2-3                             [32, 64, 56, 56]          64
├─Conv_block: 1-2                             [32, 64, 56, 56]          --
│    └─Conv2d: 2-4                            [32, 64, 56, 56]          576
│    └─BatchNorm2d: 2-5                       [32, 64, 56, 56]          128
│    └─PReLU: 2-6                             [32, 64, 56, 56]          64
├─Depth_Wise: 1-3                             [32, 64, 28, 28]          --
│    └─Conv_block: 2-7                        [32, 128, 56, 56]         --
│    │    └─Conv2d: 3-1                       [32, 128, 56, 56]         8,192
│    │    └

In [10]:
class TripletLoss(nn.Module):
    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        # calculate Euclidean's distance
        distance_positive = (anchor - positive).pow(2).sum(1)
        distance_negative = (anchor - negative).pow(2).sum(1)
        # distance_positive = F.pairwise_distance(anchor, positive).view((-1, 1))
        # distance_negative = F.pairwise_distance(anchor, negative).view((-1, 1))
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return torch.mean(losses)

In [11]:
def calculate_accuracy(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))

    acc = float(tp + tn) / dist.size

    return acc

In [12]:
model_config = {
    'margin': 1.2,
    'lr': 1e-3,
    'patience': 5,
    'factor': 0.1,
    'min_lr': 1e-10,
    'threshold': 1e-3
}
criterion = TripletLoss(margin = model_config['margin'])
optimizer = optim.Adam(triplet_model.parameters(), lr=model_config['lr'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=model_config['patience'], factor=model_config['factor'], min_lr=model_config['min_lr'], threshold=model_config['threshold'], verbose=True)

In [13]:
from statistics import mean

def train_model(model, train_loader, train_optimizer, train_criterion):
    train_loss = 0
    train_accuracy = list()
    # set to train mode
    model.train()
    # enable grad
    with torch.set_grad_enabled(True):
        # loop though loader
        for anchor_img, pos_img, neg_img in train_loader:
            # for calculate accuracy
            threshold_list = np.full(anchor_img.shape[0], model_config['margin'])
            is_same = np.full(anchor_img.shape[0], 1)

            # throw img to CUDA
            anchor_img, pos_img,neg_img = anchor_img.to(device), pos_img.to(device),neg_img.to(device)
            # prevent grad vansishing
            train_optimizer.zero_grad()
            # training model
            anchor_img, pos_img, neg_img = model(anchor_img, pos_img,neg_img)
            # calculate loss
            loss = train_criterion(anchor_img, pos_img, neg_img)
            # backpropragation
            loss.backward()
            # compute gradient
            train_optimizer.step()
            # compute train loss
            train_loss += loss.item() * anchor_img.size(0)
            # calculate distance
            distance_positive = (anchor_img - pos_img).pow(2).sum(1)
            # compute accuracy
            train_accuracy.append(calculate_accuracy(threshold_list,distance_positive.cpu().data.numpy(),is_same))

    return  train_loss, mean(train_accuracy)

def test_model(model, test_loader, test_optimizer, test_criterion):
    test_loss = 0
    test_accuracy = list()
    # set to train mode
    model.eval()
    # enable grad
    with torch.no_grad():
        # loop though loader
        for anchor_img, pos_img, neg_img in test_loader:
            # for calculate accuracy
            threshold_list = np.full(anchor_img.shape[0], model_config['margin'])
            is_same = np.full(anchor_img.shape[0], 1)

            # throw img to CUDA
            anchor_img, pos_img,neg_img = anchor_img.to(device), pos_img.to(device),neg_img.to(device)
            # training model
            anchor_img, pos_img, neg_img = model(anchor_img, pos_img,neg_img)
            # calculate loss
            loss = test_criterion(anchor_img, pos_img, neg_img)
            test_optimizer.zero_grad()
            # compute train loss
            test_loss += loss.item() * anchor_img.size(0)
            # calculate distance
            distance_positive = (anchor_img - pos_img).pow(2).sum(1)
            # compute accuracy
            test_accuracy.append(calculate_accuracy(threshold_list,distance_positive.cpu().data.numpy(),is_same))

    return test_loss, mean(test_accuracy)


In [14]:
# num_epoch = 120
# previous_lr = 0
# save_weight_path = './weights/triplet_mobilenetwork_120_epochs.pth'
# history_train = {
#     'train_loss': list(),
#     'val_loss': list(),
#     'train_acc': list(),
#     'val_acc': list()
# }
# triplet_model.to(device)
# for epoch in tqdm(range(num_epoch)):
#     # train model
#     epoch_train_loss, train_acc = train_model(triplet_model, train_triplet_dataloader, optimizer, criterion)
#     # validate model
#     epoch_val_loss, val_acc = test_model(triplet_model, val_triplet_dataloader, optimizer, criterion)
#     # compute loss
#     train_loss = epoch_train_loss / len(train_triplet_dataloader.sampler)
#     val_loss = epoch_val_loss / len(val_triplet_dataloader.sampler)
#     # update scheduler
#     scheduler.step(train_loss)

#     # append historical data
#     history_train['train_loss'].append(train_loss)
#     history_train['val_loss'].append(val_loss)

#     history_train['train_acc'].append(train_acc)
#     history_train['val_acc'].append(val_acc)

#     # get the learning rate
#     optim_lr = optimizer.param_groups[0]['lr']
#     if (optim_lr < previous_lr) | (optim_lr > previous_lr):
#         print('LEARNING RATE HAS CHANGED!')
#     # update learning rate to temp variable
#     previous_lr = optim_lr
#     # observe status
#     print(f'Epoch {epoch+1}/{num_epoch},Learning rate: {optim_lr}, train loss: {round(train_loss,5)}, valid loss: {round(val_loss,5)}, train acc: {round(train_acc,5)}, valid acc: {round(val_acc,5)}')

# # save model weight
# torch.save(triplet_model.state_dict(), save_weight_path)
# # clear GPU memory allocated
# torch.cuda.empty_cache()

In [15]:
plt.plot(np.arange(1,len(history_train['train_loss'])+1), history_train['train_loss'],label = 'Train Loss')
plt.plot(np.arange(1,len(history_train['val_loss'])+1),history_train['val_loss'], label= 'Validation loss')
plt.legend()
plt.show()

NameError: name 'history_train' is not defined

In [ ]:
plt.plot(np.arange(1,len(history_train['train_acc'])+1), history_train['train_acc'],label = 'Train Loss')
plt.plot(np.arange(1,len(history_train['val_acc'])+1),history_train['val_acc'], label= 'Validation loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(1,len(history_train['train_loss'])+1), history_train['train_loss'],label = 'Train Loss')
plt.plot(np.arange(1,len(history_train['val_loss'])+1),history_train['val_loss'], label= 'Validation loss')
plt.plot(np.arange(1,len(history_train['train_acc'])+1), history_train['train_acc'],label = 'Train Loss')
plt.plot(np.arange(1,len(history_train['val_acc'])+1),history_train['val_acc'], label= 'Validation loss')
plt.legend()
plt.show()

In [ ]:
torch.cuda.empty_cache()

In [16]:
# define transformation for test set
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(img_size),
    transforms.RandomHorizontalFlip(p=0.6),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
cross_val_train_dataset = FaceDataset(df = train_df,num_sample = 9000, transform = train_transform)
# cross_val_val_dataset = FaceDataset(df = val_df,num_sample = 5000, transform = train_transform)

  0%|          | 0/9000 [00:00<?, ?it/s]

In [17]:
num_epoch = 120
previous_lr = 0
history_train = {
    'train_loss': dict(),
    'val_loss': dict(),
    'train_acc': dict(),
    'val_acc': dict()
}
split = 3 
k_fold = KFold(n_splits = split, shuffle = True, random_state=seed)
batch_size = 64
for fold, (train_idx, valid_idx) in tqdm(enumerate(k_fold.split(cross_val_train_dataset)),total = split):

    triplet_model = MobileFaceNet()
    
    triplet_model.to(device)
    model_config = {
    'margin': 1.2,
    'lr': 1e-3,
    'patience': 5,
    'factor': 0.1,
    'min_lr': 1e-10,
    'threshold': 1e-3
    }
    criterion = TripletLoss(margin = model_config['margin'])
    optimizer = optim.Adam(triplet_model.parameters(), lr=model_config['lr'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=model_config['patience'], factor=model_config['factor'], min_lr=model_config['min_lr'], threshold=model_config['threshold'], verbose=True)
    print(f'Fold: {fold}/{split}')

    # the number of epoch in each fold
    # Random get the sample from dataset
    train_subsampler = torch.utils.data.Subset(cross_val_train_dataset,train_idx)
    valid_subsampler = torch.utils.data.Subset(cross_val_train_dataset,valid_idx)

    # Load data into dataloader
    train_loader = DataLoader(train_subsampler, batch_size = batch_size,  shuffle = True,pin_memory=True)
    valid_loader = DataLoader(valid_subsampler, batch_size = batch_size, shuffle = True, pin_memory=True)
    
    hist_epoch_train_loss = list()
    hist_epoch_val_loss = list()
    hist_epoch_train_acc = list()
    hist_epoch_val_acc = list()
    
    for epoch in tqdm(range(num_epoch)):
        # train model
        epoch_train_loss, train_acc = train_model(triplet_model, train_loader, optimizer, criterion)
        # validate model
        epoch_val_loss, val_acc = test_model(triplet_model, valid_loader, optimizer, criterion)
        # compute loss
        train_loss = epoch_train_loss / len(train_loader.sampler)
        val_loss = epoch_val_loss / len(valid_loader.sampler)
        # update scheduler
        scheduler.step(train_loss)
        
        hist_epoch_train_loss.append(train_loss)
        hist_epoch_val_loss.append(val_loss)
        
        hist_epoch_train_acc.append(train_acc)
        hist_epoch_val_acc.append(val_acc)

        # get the learning rate
        optim_lr = optimizer.param_groups[0]['lr']
        if (optim_lr < previous_lr) | (optim_lr > previous_lr):
            print('LEARNING RATE HAS CHANGED!')
        # update learning rate to temp variable
        previous_lr = optim_lr
        # observe status
        print(f'Fold: {fold+1}/{split}, Epoch {epoch+1}/{num_epoch},Learning rate: {optim_lr}, train loss: {round(train_loss,5)}, valid loss: {round(val_loss,5)}, train acc: {round(train_acc,5)}, val acc: {round(val_acc,5)}')
    
    # append historical data
    history_train['train_loss'][fold] = hist_epoch_train_loss
    history_train['val_loss'][fold] = hist_epoch_val_loss

    history_train['train_acc'][fold] = hist_epoch_train_acc
    history_train['val_acc'][fold] = hist_epoch_val_acc
    
    print(f'Average result {fold+1}/{split}, Avg train loss:', mean(history_train['train_loss'][fold]), 'Avg Val loss:',mean(history_train['val_loss'][fold]),'avg train acc:', mean(history_train['train_acc'][fold]), 'avg val acc:', mean(history_train['val_acc'][fold]) )
    # clear GPU memory allocated
    torch.cuda.empty_cache()

  0%|          | 0/3 [00:00<?, ?it/s]

Fold: 0/3


  0%|          | 0/120 [00:00<?, ?it/s]

LEARNING RATE HAS CHANGED!
Fold: 1/3, Epoch 1/120,Learning rate: 0.001, train loss: 0.75852, valid loss: 0.59034, train acc: 0.43711, val acc: 0.68247
Fold: 1/3, Epoch 2/120,Learning rate: 0.001, train loss: 0.60778, valid loss: 0.50149, train acc: 0.63104, val acc: 0.72749
Fold: 1/3, Epoch 3/120,Learning rate: 0.001, train loss: 0.54559, valid loss: 0.4663, train acc: 0.67099, val acc: 0.77508
Fold: 1/3, Epoch 4/120,Learning rate: 0.001, train loss: 0.50049, valid loss: 0.45728, train acc: 0.7157, val acc: 0.7747
Fold: 1/3, Epoch 5/120,Learning rate: 0.001, train loss: 0.48204, valid loss: 0.46392, train acc: 0.72883, val acc: 0.75418
Fold: 1/3, Epoch 6/120,Learning rate: 0.001, train loss: 0.44733, valid loss: 0.39774, train acc: 0.75482, val acc: 0.79555
Fold: 1/3, Epoch 7/120,Learning rate: 0.001, train loss: 0.4321, valid loss: 0.41404, train acc: 0.76629, val acc: 0.79483
Fold: 1/3, Epoch 8/120,Learning rate: 0.001, train loss: 0.41404, valid loss: 0.39268, train acc: 0.78258, va

Fold: 1/3, Epoch 67/120,Learning rate: 0.001, train loss: 0.19981, valid loss: 0.19686, train acc: 0.91728, val acc: 0.92962
Fold: 1/3, Epoch 68/120,Learning rate: 0.001, train loss: 0.18896, valid loss: 0.18322, train acc: 0.92448, val acc: 0.9394
Fold: 1/3, Epoch 69/120,Learning rate: 0.001, train loss: 0.18627, valid loss: 0.17459, train acc: 0.92559, val acc: 0.93669
Fold: 1/3, Epoch 70/120,Learning rate: 0.001, train loss: 0.19025, valid loss: 0.18701, train acc: 0.92675, val acc: 0.93042
Fold: 1/3, Epoch 71/120,Learning rate: 0.001, train loss: 0.19998, valid loss: 0.18356, train acc: 0.9257, val acc: 0.93769
Fold: 1/3, Epoch 72/120,Learning rate: 0.001, train loss: 0.19335, valid loss: 0.17404, train acc: 0.92963, val acc: 0.93465
Fold: 1/3, Epoch 73/120,Learning rate: 0.001, train loss: 0.18851, valid loss: 0.17884, train acc: 0.93102, val acc: 0.9337
Fold: 1/3, Epoch 74/120,Learning rate: 0.001, train loss: 0.19014, valid loss: 0.16895, train acc: 0.92692, val acc: 0.94078
Epo

...

LEARNING RATE HAS CHANGED!
Fold: 2/3, Epoch 1/120,Learning rate: 0.001, train loss: 0.79725, valid loss: 0.65465, train acc: 0.42836, val acc: 0.66024
Fold: 2/3, Epoch 2/120,Learning rate: 0.001, train loss: 0.6049, valid loss: 0.54427, train acc: 0.61691, val acc: 0.69139
Fold: 2/3, Epoch 3/120,Learning rate: 0.001, train loss: 0.53469, valid loss: 0.5432, train acc: 0.67769, val acc: 0.76368
Fold: 2/3, Epoch 4/120,Learning rate: 0.001, train loss: 0.50086, valid loss: 0.50304, train acc: 0.70994, val acc: 0.75527
Fold: 2/3, Epoch 5/120,Learning rate: 0.001, train loss: 0.48486, valid loss: 0.4038, train acc: 0.72407, val acc: 0.82456
Fold: 2/3, Epoch 6/120,Learning rate: 0.001, train loss: 0.47531, valid loss: 0.41287, train acc: 0.73432, val acc: 0.76567
Fold: 2/3, Epoch 7/120,Learning rate: 0.001, train loss: 0.43143, valid loss: 0.38141, train acc: 0.75754, val acc: 0.79939
Fold: 2/3, Epoch 8/120,Learning rate: 0.001, train loss: 0.41254, valid loss: 0.39008, train acc: 0.76779, v

Fold: 2/3, Epoch 66/120,Learning rate: 1e-05, train loss: 0.16698, valid loss: 0.16797, train acc: 0.93639, val acc: 0.95327
Fold: 2/3, Epoch 67/120,Learning rate: 1e-05, train loss: 0.17146, valid loss: 0.15874, train acc: 0.93966, val acc: 0.95597
Fold: 2/3, Epoch 68/120,Learning rate: 1e-05, train loss: 0.16675, valid loss: 0.14427, train acc: 0.94498, val acc: 0.95939
Fold: 2/3, Epoch 69/120,Learning rate: 1e-05, train loss: 0.17272, valid loss: 0.14144, train acc: 0.94082, val acc: 0.96068
Fold: 2/3, Epoch 70/120,Learning rate: 1e-05, train loss: 0.16632, valid loss: 0.14226, train acc: 0.9401, val acc: 0.95868
Fold: 2/3, Epoch 71/120,Learning rate: 1e-05, train loss: 0.16995, valid loss: 0.15444, train acc: 0.94138, val acc: 0.9564
Fold: 2/3, Epoch 72/120,Learning rate: 1e-05, train loss: 0.17064, valid loss: 0.14837, train acc: 0.94188, val acc: 0.95654
Fold: 2/3, Epoch 73/120,Learning rate: 1e-05, train loss: 0.17277, valid loss: 0.15412, train acc: 0.93999, val acc: 0.95232
Fo

  0%|          | 0/120 [00:00<?, ?it/s]

LEARNING RATE HAS CHANGED!
Fold: 3/3, Epoch 1/120,Learning rate: 0.001, train loss: 0.77313, valid loss: 0.59661, train acc: 0.43057, val acc: 0.64765
Fold: 3/3, Epoch 2/120,Learning rate: 0.001, train loss: 0.59093, valid loss: 0.50584, train acc: 0.64256, val acc: 0.73893
Fold: 3/3, Epoch 3/120,Learning rate: 0.001, train loss: 0.55501, valid loss: 0.4815, train acc: 0.67814, val acc: 0.75898
Fold: 3/3, Epoch 4/120,Learning rate: 0.001, train loss: 0.50982, valid loss: 0.40733, train acc: 0.7244, val acc: 0.7889
Fold: 3/3, Epoch 5/120,Learning rate: 0.001, train loss: 0.48351, valid loss: 0.36403, train acc: 0.73532, val acc: 0.81464
Fold: 3/3, Epoch 6/120,Learning rate: 0.001, train loss: 0.46305, valid loss: 0.37232, train acc: 0.73166, val acc: 0.79493
Fold: 3/3, Epoch 7/120,Learning rate: 0.001, train loss: 0.43745, valid loss: 0.41352, train acc: 0.75205, val acc: 0.77726
Fold: 3/3, Epoch 8/120,Learning rate: 0.001, train loss: 0.41212, valid loss: 0.33543, train acc: 0.76995, v

Fold: 3/3, Epoch 67/120,Learning rate: 0.001, train loss: 0.19439, valid loss: 0.16645, train acc: 0.9273, val acc: 0.95042
Fold: 3/3, Epoch 68/120,Learning rate: 0.001, train loss: 0.20338, valid loss: 0.17221, train acc: 0.92348, val acc: 0.94695
Fold: 3/3, Epoch 69/120,Learning rate: 0.001, train loss: 0.19323, valid loss: 0.17571, train acc: 0.92681, val acc: 0.95132
Fold: 3/3, Epoch 70/120,Learning rate: 0.001, train loss: 0.19151, valid loss: 0.16698, train acc: 0.9288, val acc: 0.95564
Fold: 3/3, Epoch 71/120,Learning rate: 0.001, train loss: 0.18774, valid loss: 0.1611, train acc: 0.93318, val acc: 0.95104
Fold: 3/3, Epoch 72/120,Learning rate: 0.001, train loss: 0.18507, valid loss: 0.17181, train acc: 0.93196, val acc: 0.94301
Fold: 3/3, Epoch 73/120,Learning rate: 0.001, train loss: 0.19495, valid loss: 0.17608, train acc: 0.92758, val acc: 0.94766
Fold: 3/3, Epoch 74/120,Learning rate: 0.001, train loss: 0.20198, valid loss: 0.17067, train acc: 0.92027, val acc: 0.94548
Fol

In [18]:
# save_weight_path = './weights/triplet_mobilenetwork_120_epochs_updated.pth'
# torch.save(triplet_model.state_dict(), save_weight_path)

In [24]:
history_train['train_loss'][1]

[0.7972512998580933,
 0.6049048628807068,
 0.5346884415944417,
 0.5008646562894186,
 0.48485850024223326,
 0.4753139651616414,
 0.4314347077210744,
 0.41254191573460897,
 0.4007680857976278,
 0.38485235516230265,
 0.36948317917188006,
 0.3744330619176229,
 0.3565329434076945,
 0.3532071644862493,
 0.33323392264048257,
 0.3266823902130127,
 0.3215738417307536,
 0.3206024248600006,
 0.3173646051088969,
 0.31414996067682904,
 0.3080346000989278,
 0.30961345219612124,
 0.311389554977417,
 0.3034162600040436,
 0.284575403769811,
 0.2820958143870036,
 0.28547751609484356,
 0.2920718656380971,
 0.2845785083770752,
 0.2674882452090581,
 0.2632804310321808,
 0.26567713912328084,
 0.26115509470303855,
 0.25306651544570924,
 0.23580067678292593,
 0.2561314174334208,
 0.24310588383674622,
 0.23804169209798176,
 0.24368229059378307,
 0.23581870206197103,
 0.23897733688354492,
 0.21434970752398172,
 0.20601265541712444,
 0.19628245202700298,
 0.20145717708269756,
 0.20277032848199208,
 0.20387338234